# SYCL devices

In [ ]:
%%writefile compute.cpp
#include <iostream>
#include <sycl/sycl.hpp>
#include <vector>

class vector_dot_product_cpu;
//using namespace sycl;
int main() {
    const int N = 2;              // Length of the vectors
    std::vector<int> vectorA(N, 1);  // Vector A filled with 1s
    std::vector<int> vectorB(N, 2);  // Vector B filled with 2s
    int result = 0;  // Result of dot product initialized to zero

    sycl::queue cpuQueue(sycl::default_selector_v);
    std::cout << "Running on "
              << cpuQueue.get_device().get_info<sycl::info::device::name>()
              << std::endl;

   // auto cpuQueue = sycl::queue{sycl::gpu_selector_v};
    
    auto bufA = sycl::buffer{vectorA.data(), sycl::range{N}};
    auto bufB = sycl::buffer{vectorB.data(), sycl::range{N}};
    auto bufResult = sycl::buffer{&result, sycl::range{1}};

    cpuQueue.submit([&](sycl::handler& cgh) {
            auto accA = bufA.get_access<sycl::access::mode::read>(cgh);
            auto accB = bufB.get_access<sycl::access::mode::read>(cgh);
            auto accResult =
                bufResult.get_access<sycl::access::mode::read_write>(
                    cgh);

            cgh.parallel_for<vector_dot_product_cpu>(
                sycl::range{N}, [=](sycl::id<1> idx) {
                    //int i = idx[0];
                    accResult[0] += accA[idx] * accB[idx];
                });
        })
        .wait();

    std::cout << "Dot product completed on CPU. Result: " << result
              << std::endl;
    return 0;
}


In [ ]:
%%writefile ./run-sycl.sh

#!/bin/bash 
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

In [ ]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh

In [ ]:
%%writefile compute.cpp
#include <iostream>
#include <sycl/sycl.hpp>
#include <vector>

//class vector_dot_product_gpu;

int main() {
    const int N = 2;              // Length of the vectors
    std::vector<int> vectorA(N, 1);  // Vector A filled with 1s
    std::vector<int> vectorB(N, 2);  // Vector B filled with 2s
    int result = 0;  // Result of dot product initialized to zero

    auto gpuQueue = sycl::queue{sycl::cpu_selector_v};

    auto bufA = sycl::buffer{vectorA.data(), sycl::range{N}};
    auto bufB = sycl::buffer{vectorB.data(), sycl::range{N}};
    auto bufResult = sycl::buffer{&result, sycl::range{1}};

    

    gpuQueue.submit([&](sycl::handler& cgh) {
            auto accA = bufA.get_access<sycl::access::mode::read>(cgh);
            auto accB = bufB.get_access<sycl::access::mode::read>(cgh);
            auto accResult = bufResult.get_access<sycl::access::mode::read_write>(cgh);
            sycl::stream sout(10240, 2560, cgh);
            

            cgh.parallel_for<class vector_dot_product_gpu>(
                sycl::range{N}, [=](sycl::id<1> idx) {
                int i = idx[0];
                sout << "id global " << idx[0] << sycl::endl;
                sout << "product = " << accA[i] * accB[i] << sycl::endl;
                    
                //accResult[0] = 99;
                accResult[0] += accA[i] * accB[i];
                sout << "dot = " << accResult[0] << sycl::endl;
                });
        })
        .wait();

    std::cout << "Dot product completed successfully on GPU. Result: " << result
              << std::endl;
    return 0;
}


In [ ]:
%%writefile ./run-sycl.sh

#!/bin/bash -x
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

In [ ]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh